In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import networkx as nx

from pyqubo import Binary, Array
from pprint import pprint
from qat.lang.AQASM import Program, QRoutine, RZ, RX, CNOT, H, X
from qat.core import Observable, Term
from qat.qpus import get_default_qpu
from qat.plugins import ScipyMinimizePlugin

# Problema da Cobertura Exata (Exact Cover Problem)

Considere um conjunto $S = \{1, \ldots, n\}$ e uma coleção de subconjuntos.
$$ C = \{C_1, C_2, \ldots, C_n\} \subseteq 2^S$$
Uma cobertura de S com os conjuntos da coleção C é definida como o conjunto dos conjuntos $C_j, \ldots, C_k \in C$ tal que
$$\bigcup_i V_i = S$$
Para uma cobertura ser exata, os conjuntos que definem a cobertura devem ser disjuntos.


In [ ]:
#U = [1,2,3,4,5,6]
#V = [{1,2,3}, {4,5,6}, {1}, {2}, {3}, {4}, {5}, {6}]
U = [1,2,3,4,5,6,7]
V = [{1,7}, {1,4,7}, {1,4}, {4,5,7}, {3,5,6}, {2,3,6}, {2,7}]
A = 8
B = 1.5

## Hamiltoniano

Exact Cover Hamiltonian
$$ H_P = \sum^n_{\alpha=1} (1 - \sum_{i:\alpha\in V_i} x_i)^2$$

Min Exact Cover Hamiltonian
$$ H_P = A \sum^n_{\alpha=1} (1 - \sum_{i:\alpha\in V_i} x_i)^2 + B\sum_i x_i$$
$$A > nB$$

In [ ]:
variables = Array.create('x', shape=len(V), vartype='BINARY')
print(variables)

In [ ]:
HA = 0
for alpha in U:
    temp = sum(variables[i] for i, Vi in enumerate(V) if alpha in Vi)
    HA += (1-temp)**2
HA *= A

print(HA)

In [ ]:
HB = B * sum(x for x in variables)

print(HB)

In [ ]:
Hamil = HA + HB
print(Hamil)

## Modelar como QUBO

$$-\sum_i^N h_ix_i - \sum_{i<j} J_{ij} x_ix_j $$

In [ ]:
model = Hamil.compile()
qubo = model.to_qubo()
pprint(qubo)

In [ ]:
single_values = {}
for k,v in qubo[0].items():
    if k[0] == k[1]:
        single_values[int(k[0][2])] = v

multiple_values = {}
for k,v in qubo[0].items():
    if k[0] != k[1]:
        multiple_values[(int(k[0][2]), int(k[1][2]))] = v

num_qubits = len(variables)
hamiltonian_exact_cover = Observable(num_qubits,
                           pauli_terms=
                           [Term(single_values[x], "Z", [x]) for x in single_values]+
                           [Term(multiple_values[x], "ZZ", [x[0],x[1]]) for x in multiple_values],
                           constant_coeff=qubo[1]
                           )

print(hamiltonian_exact_cover)

## QAOA Ansatz

### Separador de Fase

In [ ]:
def phase_separator_exact_cover(qreg, gamma, single_values, multiple_values):
    for i in single_values:
        RZ(gamma)(qreg[i])
    for j in multiple_values:
        CNOT(qreg[j[0]], qreg[j[1]])
        RZ(2*gamma)(qreg[j[1]])
        CNOT(qreg[j[0]], qreg[j[1]])

### Misturador (Mixer)

In [ ]:
def mixer_exact_cover(qreg, beta, num_qubits):
        for q in range(num_qubits):
            RX(beta)(qreg[q])

In [ ]:
# --------------------------
# Running QAOA for Max Independent Set on simulator
# --------------------------

# -------------------
# Initializing qubits
# -------------------
p = 4
qprog = Program()

gamma  = [qprog.new_var(float, '\\gamma_%s'%i) for i in range(1,p+1)]
beta   = [qprog.new_var(float, '\\beta_%s'%i) for i in range(1,p+1)]

# --------------------------
# Initial state preparation
# --------------------------
qbits = qprog.qalloc(num_qubits)
for q in range(num_qubits):
    H(qbits[q])

# ----------------------------------
# Alternate application of operators
# ----------------------------------
for step in range(p):
    phase_separator_exact_cover(qbits, gamma[step], single_values, multiple_values)
    mixer_exact_cover(qbits, beta[step], num_qubits)

### Visualização do Circuito

In [ ]:
circuit = qprog.to_circ()
print("total number of gates: ", len(circuit.ops))
print("Variables:", circuit.get_variables())
# Display quantum circuit
%qatdisplay circuit --svg

## Rodar o Circuito usando QLM

In [ ]:
# Create a job
job = circuit.to_job(observable=hamiltonian_exact_cover)

result_list = []
for _ in range(10):
    ## A cobyla minimizer over any number of variables, random initialization, 200 max steps
    cobyla = ScipyMinimizePlugin(tol=1e-6,
                                method="COBYLA",
                                options={"maxiter": 300},
                                x0=[random.uniform(0, 2*np.pi) for _ in range(2*p)])
    # Create a Quantum Processor Unit
    qpu = get_default_qpu()

    stack = cobyla | qpu

    # Submit the job to the QPU
    result_list.append(stack.submit(job))

## Análise dos Resultados

In [ ]:
for i, r in enumerate(result_list):
    print("Run", i, ", Final energy:", r.value)
    #Binding the variables:random.uniform(0, 2*np.pi)
    sol_job = job(**eval(r.meta_data["parameter_map"]))

    #Rerunning in 'SAMPLE' mode to get the most probable states:
    sampling_job = sol_job.circuit.to_job()

    sol_res = qpu.submit(sampling_job)
    print("Most probable states are:")
    for sample in sol_res:
        if sample.probability > 0.05:
            print(sample.state, "{:.2f}%".format(100 * sample.probability))

In [ ]:
result = min(result_list, key=lambda s: s.value)
print("Final energy:", result.value)
for key, value in result.meta_data.items():
    print(key, ":", value)

In [ ]:
plt.plot(eval(result.meta_data["optimization_trace"]))
plt.xlabel("steps")
plt.ylabel("energy")
plt.show()

In [ ]:
import itertools
def exact_cover_obj(result, U, V):
    solution = [Vi for i, Vi in enumerate(V) if result[i] == '1']
    for i in itertools.combinations(solution, 2):
        if len(i[0].intersection(i[1])) != 0:
            return np.inf
    cover = set().union(*solution)
    difference = set(U) - cover
    if len(difference) == 0:
        return len(solution)
    else:
        return np.inf

In [ ]:
# Emulating a reasonnable setup:
# Drawing 1024 cuts
sol_job = job(**eval(result.meta_data["parameter_map"]))
sampling_job = sol_job.circuit.to_job(nbshots=1024)
sol_res = qpu.submit(sampling_job)

tuple_list = [(s.state.value[0], exact_cover_obj(s.state.value[0],U,V)) for s in sol_res]

exact_cover = min(tuple_list, key=lambda s: s[1])[0]
print(
    "Exact Cover:",
    [Vi for i, Vi in enumerate(V) if exact_cover[i] == '1']
)